In [4]:
import torch
import numpy as np
import torch.nn as nn
import math
import os
import pandas as pd
import torch.optim as optim
import torch.nn.functional as F
import json
from torch import nn, Tensor

MAX_LENGTH = 520
tokens = json.load(open("tokens.txt"))
model = torch.load("embedding_model")
file_path = os.path.join(os.path.expanduser("~"), "Downloads", "mental_health.csv")
orig_dataset = pd.read_csv(file_path)
orig_dataset = orig_dataset.to_numpy()
trainingSet =  orig_dataset[np.random.choice(orig_dataset.shape[0], 4, replace=True)] #extract training set
contextSet = [trainingSet[i][0] for i in range(len(trainingSet))] 
responseSet = [trainingSet[i][1] for i in range(len(trainingSet))]
#tokenizing the context and response set, also 0 is special token for unknown word
contextSet_tokenized = [[tokens[word] if word in tokens else 0 for word in example.split()] 
                         for example in contextSet ]
responseSet_tokenized = [[tokens[word] if word in tokens else 0 for word in example.split()] 
                         for example in responseSet ]

#convert token to input embedding for context and response set filled with padding
#tokens if end of sentence. 1 is special token for padding 
contextSet_embedding = []        
for context in contextSet_tokenized:
    contextEmbedding = []
    for i in range(MAX_LENGTH):
        if i>= len(context):
            contextEmbedding.append(model["embeddings.weight"][1])
            continue
        contextEmbedding.append(model["embeddings.weight"][context[i]])
    contextEmbedding = torch.stack(contextEmbedding)
    contextSet_embedding.append(contextEmbedding[:])

contextSet_embedding = torch.stack(contextSet_embedding)

responseSet_embedding = []        
for response in responseSet_tokenized:
    responseEmbedding = []
    for i in range(MAX_LENGTH):
        if i>= len(response):
            responseEmbedding.append(model["embeddings.weight"][1])
            continue
        responseEmbedding.append(model["embeddings.weight"][response[i]])
    responseEmbedding = torch.stack(responseEmbedding)
    responseSet_embedding.append(responseEmbedding[:])
    
responseSet_embedding = torch.stack(responseSet_embedding)
print(responseSet_embedding.shape)


torch.Size([4, 520, 252])


In [5]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [6]:
pe = PositionalEncoding(252, max_len = MAX_LENGTH)
pe.forward(contextSet_embedding)

tensor([[[-3.7979,  0.9362, -0.0986,  ..., -0.3799,  0.0000, -0.0308],
         [ 0.6926,  1.7718,  0.4771,  ..., -0.5516, -1.2003,  1.9489],
         [-0.2971,  1.0521, -0.0362,  ...,  3.5024,  0.3547, -0.2309],
         ...,
         [ 0.0715,  1.0004,  1.2058,  ...,  2.0074,  0.7355,  1.0923],
         [ 0.0715,  1.0004,  1.2058,  ...,  0.0000,  0.7355,  1.0923],
         [ 0.0715,  0.0000,  1.2058,  ...,  2.0074,  0.7355,  1.0923]],

        [[-0.3776,  0.8181, -1.9062,  ...,  0.0000,  1.0079,  0.0000],
         [ 0.7852,  1.5502,  1.5748,  ...,  2.7400, -1.3242,  0.5876],
         [ 0.5004,  2.2827,  0.0000,  ...,  0.6154,  2.5610,  1.1702],
         ...,
         [ 1.0065,  0.4896,  2.0961,  ...,  0.0000,  0.7356,  1.0923],
         [ 1.0065,  0.4896,  2.0961,  ...,  2.0074,  0.7356,  1.0923],
         [ 1.0065,  0.4896,  0.0000,  ...,  2.0074,  0.7356,  1.0923]],

        [[-0.3022, -0.2446, -1.7313,  ...,  0.5140,  1.0080,  0.3587],
         [ 1.8306, -1.6114,  2.1992,  ...,  1

In [ ]:
class MultiHeadAttentionBlock(nn.module()):
    def __init__(init_(self, d_model: int, h: int, dropout: float)->None:
        self.d_model = d_model # Embedding vector size
        self.h = h # Number of heads
        # Make sure d_model is divisible by h
        assert d_model % h == 0, "d_model is not divisible by h"
        self.d_k = d_model // h # Dimension of vector seen by each head
        self.w_q = nn.Linear(d_model, d_model, bias=False) # linear layer for queue
        self.w_k = nn.Linear(d_model, d_model, bias=False) # liner layer for key
        self.w_v = nn.Linear(d_model, d_model, bias=False) # linear layer for 
        self.w_o = nn.Linear(d_model, d_model, bias=False) # Wo
        self.dropout = nn.Dropout(dropout)
                 
    @staticmethod
    def attention(query, key, value, mask, dropout: nn.Dropout):
        #d_k is number of embeddings per word for each head
        d_k = query.shape[-1]
        # Just apply the formula from the paper
        # (batch, h, seq_len, d_k) --> (batch, h, seq_len, seq_len)
        # [[[[4,3],[2,1]],[[2, 5], [3,2]]]] * [[[[4,3][2,5]],[[2,1], [3,2]]]] = 
        #[[[[0.7, 0.3], [0.1, 0.9]], [0.8, 0.2], [0.4, 0.6]]]
        #basically saying the first word of the first half of the embedding(for head 1) is related to
        #to its own word by 0.7 and related to the first half embedding of the second word by 0.3
        #also saying the second word of the first half of the embedding(for head 1) is related to
        #to its own word by 0.9 and related to the first half embedding of the first word by 0.1
        #essentially a attention matrix for each part of the embedding for the head
        attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)
        if mask is not None:
            # Write a very low value (indicating -inf) to the positions where mask == 0
            attention_scores.masked_fill_(mask == 0, -1e9)
        attention_scores = attention_scores.softmax(dim=-1) # (batch, h, seq_len, seq_len) # Apply softmax
        if dropout is not None:
            attention_scores = dropout(attention_scores)
        # (batch, h, seq_len, seq_len) --> (batch, h, seq_len, d_k)
        # return attention scores which can be used for visualization
        return (attention_scores @ value), attention_scores
    
    def forward(self, q, k, v, mask):
        query = self.w_q(q) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)
        key = self.w_k(k) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)
        value = self.w_v(v) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)
        # (batch, seq_len, d_model) --> (batch, seq_len, h, d_k) --> (batch, h, seq_len, d_k)
        #[[[4,3,2,1] , [2,5,3,2]]]
        #3rd dimension represents word embeddings of a word
        #2nd dimension represent each word embedding in an example
        #1st dimension represents each example containing each word embedding
        # * Transform the dimensions using .view *
        # [[[[4,3],[2,1]],[[2, 5], [3,2]]]]
        #4th dimenension represents the size of the split embeddings for each multi-head attention
        #3rd dimension represents the number of heads. Note that head times the size of split embeddings equals original embedding dimension
        #2nd dimension represents sequence length
        #1st dimension represents example lenght
        # * Transponse dimensions 1,2 *
        # [[[[4,3][2,5]],[[2,1], [3,2]]]]
        # 4th dimension represents the size of the split embeddings for each multi-head attention
        # 3rd dimension represents the length of the sequence. Contains each word with its associated head
        # 2nd dimension represents the number of heads
        # 1st dimension is the number of examples
        #[[[[word 1 with first half embeddings][word 2 with first half embedding]],[[word 1 with second half], [word 2 with second half]]]]        
        query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1, 2)
        key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1, 2)
        value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1, 2)
        # Calculate attention
        x, self.attention_scores = MultiHeadAttentionBlock.attention(query, key, value, mask, self.dropout)
        # Combine all the heads together
        # (batch, h, seq_len, d_k) --> (batch, seq_len, h, d_k) --> (batch, seq_len, d_model)
        x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.h * self.d_k)
        #pass combined heads to last linear layer
        return self.w_o(x)



In [ ]:
class FeedForwardBlock(nn.Module):
    def __init__(self, d_model: int, d_ff: int, dropout: float) -> None:
        super().__init__()
        self.linear_1 = nn.Linear(d_model, d_ff) # w1 and b1
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model) # w2 and b2

    def forward(self, x):
        # (batch, seq_len, d_model) --> (batch, seq_len, d_ff) --> (batch, seq_len, d_model)
        return self.linear_2(self.dropout(torch.relu(self.linear_1(x)))

In [ ]:
class ResidualConnection(nn.Module):
    
    def __init__(self, features: int, dropout: float) -> None:
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.norm = LayerNormalization(features)
    
    def forward(self, x, sublayer):
        return x + self.dropout(sublayer(self.norm(x)))
